In [103]:
import numpy as np
from pypower.api import case9,case39, ppoption, runpf, printpf,makeYbus

In [104]:
ppc = case39()
ppopt = ppoption(PF_ALG=2)
r = runpf(ppc, ppopt)



PYPOWER Version 5.0.0, 29-May-2015 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 4 P-iterations and 3 Q-iterations.

Converged in 0.04 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             39     Total Gen Capacity    7367.0        -160.0 to 2807.0
Generators        10     On-line Capacity      7367.0        -160.0 to 2807.0
Committed Gens    10     Generation (actual)   6297.9            1274.9
Loads             21     Load                  6254.2            1387.1
  Fixed           21       Fixed               6254.2            1387.1
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches          46     Losses (I^2 * Z)        43.64           1000.59

/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/pypower/ext2int.py:152: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  o["bus"]["e2i"] = zeros(max(o["bus"]["i2e"]) + 1)


In [105]:
from sys import stderr

from numpy import ones, conj, nonzero, any, exp, pi, r_
from scipy.sparse import csr_matrix

from pypower.idx_bus import BUS_I, GS, BS
from pypower.idx_brch import F_BUS, T_BUS, BR_R, BR_X, BR_B, BR_STATUS, SHIFT, TAP


def makeYbus(baseMVA, bus, branch):
    """Builds the bus admittance matrix and branch admittance matrices.
    Returns the full bus admittance matrix (i.e. for all buses) and the
    matrices C{Yf} and C{Yt} which, when multiplied by a complex voltage
    vector, yield the vector currents injected into each line from the
    "from" and "to" buses respectively of each line. Does appropriate
    conversions to p.u.
    @see: L{makeSbus}
    @author: Ray Zimmerman (PSERC Cornell)
    """
    ## constants
    nb = bus.shape[0]          ## number of buses
    nl = branch.shape[0]       ## number of lines

    bus = np.copy(bus)
    branch = np.copy(branch)
    
    bus[:, BUS_I] = bus[:, BUS_I] - 1
    branch[:, 0] = branch[:, 0] - 1
    branch[:, 1] = branch[:, 1] - 1
    
    ## check that bus numbers are equal to indices to bus (one set of bus nums)
    
    if any(bus[:, BUS_I].astype(np.int) != list(range(nb))):
        stderr.write('buses must appear in order by bus number\n')

    ## for each branch, compute the elements of the branch admittance matrix where
    ##
    ##      | If |   | Yff  Yft |   | Vf |
    ##      |    | = |          | * |    |
    ##      | It |   | Ytf  Ytt |   | Vt |
    ##
    stat = branch[:, BR_STATUS]              ## ones at in-service branches
    Ys = stat / (branch[:, BR_R] + 1j * branch[:, BR_X])  ## series admittance
    Bc = stat * branch[:, BR_B]              ## line charging susceptance
    tap = ones(nl)                           ## default tap ratio = 1
    i = nonzero(branch[:, TAP])              ## indices of non-zero tap ratios
    tap[i] = branch[i, TAP]                  ## assign non-zero tap ratios
    tap = tap * exp(1j * pi / 180 * branch[:, SHIFT]) ## add phase shifters

    Ytt = Ys + 1j * Bc / 2
    Yff = Ytt / (tap * conj(tap))
    Yft = - Ys / conj(tap)
    Ytf = - Ys / tap

    ## compute shunt admittance
    ## if Psh is the real power consumed by the shunt at V = 1.0 p.u.
    ## and Qsh is the reactive power injected by the shunt at V = 1.0 p.u.
    ## then Psh - j Qsh = V * conj(Ysh * V) = conj(Ysh) = Gs - j Bs,
    ## i.e. Ysh = Psh + j Qsh, so ...
    ## vector of shunt admittances
    Ysh = (bus[:, GS] + 1j * bus[:, BS]) / baseMVA

    ## build connection matrices
    f = branch[:, F_BUS]                           ## list of "from" buses
    t = branch[:, T_BUS]                           ## list of "to" buses
    ## connection matrix for line & from buses
    Cf = csr_matrix((ones(nl), (range(nl), f)), (nl, nb))
    ## connection matrix for line & to buses
    Ct = csr_matrix((ones(nl), (range(nl), t)), (nl, nb))

    ## build Yf and Yt such that Yf * V is the vector of complex branch currents injected
    ## at each branch's "from" bus, and Yt is the same for the "to" bus end
    i = r_[range(nl), range(nl)]                   ## double set of row indices

    Yf = csr_matrix((r_[Yff, Yft], (i, r_[f, t])), (nl, nb))
    Yt = csr_matrix((r_[Ytf, Ytt], (i, r_[f, t])), (nl, nb))
    # Yf = spdiags(Yff, 0, nl, nl) * Cf + spdiags(Yft, 0, nl, nl) * Ct
    # Yt = spdiags(Ytf, 0, nl, nl) * Cf + spdiags(Ytt, 0, nl, nl) * Ct

    ## build Ybus
    Ybus = Cf.T * Yf + Ct.T * Yt + \
        csr_matrix((Ysh, (range(nb), range(nb))), (nb, nb))

    return Ybus, Yf, Yt

In [106]:
Ybus, Yf, Yt = makeYbus(ppc['baseMVA'],ppc['bus'],ppc['branch'])


In [107]:
Ybus.todense()

matrix([[  3.65450097 -63.36747732j,  -2.05705688 +24.15572508j,
           0.00000000  +0.j        , ...,   0.00000000  +0.j        ,
           0.00000000  +0.j        ,  -1.59744409 +39.93610224j],
        [ -2.05705688 +24.15572508j,  64.64569545-211.87056921j,
          -5.65955594 +65.73791902j, ...,   0.00000000  +0.j        ,
           0.00000000  +0.j        ,   0.00000000  +0.j        ],
        [  0.00000000  +0.j        ,  -5.65955594 +65.73791902j,
          14.69062005-186.84298941j, ...,   0.00000000  +0.j        ,
           0.00000000  +0.j        ,   0.00000000  +0.j        ],
        ..., 
        [  0.00000000  +0.j        ,   0.00000000  +0.j        ,
           0.00000000  +0.j        , ...,   1.11399926 -43.07463795j,
           0.00000000  +0.j        ,   0.00000000  +0.j        ],
        [  0.00000000  +0.j        ,   0.00000000  +0.j        ,
           0.00000000  +0.j        , ...,   0.00000000  +0.j        ,
           3.27868852 -63.93442623j,   0.000000

In [108]:
bus[:, BUS_I].astype(np.int)

array([1, 2])

In [109]:
import pypower
pypower.bustypes
pypower.makeYbus.makeYbus(ppc['baseMVA'], ppc['bus'], ppc['branch'])

buses must appear in order by bus number


ValueError: column index exceeds matrix dimensions

In [44]:
pypower.bustypes.BUS_TYPE


1

In [38]:
ppc['bus']

array([[  1.00000000e+00,   1.00000000e+00,   9.76000000e+01,
          4.42000000e+01,   0.00000000e+00,   0.00000000e+00,
          2.00000000e+00,   1.03938360e+00,  -1.35366020e+01,
          3.45000000e+02,   1.00000000e+00,   1.06000000e+00,
          9.40000000e-01],
       [  2.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.00000000e+00,   1.04849410e+00,  -9.78526660e+00,
          3.45000000e+02,   1.00000000e+00,   1.06000000e+00,
          9.40000000e-01],
       [  3.00000000e+00,   1.00000000e+00,   3.22000000e+02,
          2.40000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.00000000e+00,   1.03070770e+00,  -1.22763840e+01,
          3.45000000e+02,   1.00000000e+00,   1.06000000e+00,
          9.40000000e-01],
       [  4.00000000e+00,   1.00000000e+00,   5.00000000e+02,
          1.84000000e+02,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   1.00446000e+00,  -1.262